In [20]:
# imports
import datetime as dt
import json
import random
from itertools import combinations_with_replacement
from pathlib import Path

import numpy as np
import pandas as pd
import psycopg2
import scipy.stats as stats
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import ceil
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity

from game_lists_site.models import (
    BenchmarkUserCBR,
    BenchmarkUserMBCF,
    BenchmarkUserSimilarity,
    Game,
    System,
    User,
    UserGame,
    GameStats,
    Parameters,
    db,
    user_data_dir,
)
from game_lists_site.utilities import (
    ParametersManager,
    get_readable_result_for_games,
    merge_dicts,
    get_normalized_playtimes,
    days_delta,
    slice_dict,
    normalize_dict,
)
from game_lists_site.algorithms.game import update_cbr_for_game
from game_lists_site.algorithms.user import update_similar_users, MF, update_mobcf

db.rollback()


In [21]:
# get users
def get_users():
    users = []
    for user in User.select():
        game_with_score_count = (
            UserGame.select(UserGame.id)
            .where((UserGame.user == user) & (UserGame.score > 0))
            .count()
        )
        if game_with_score_count > 1:
            users.append(user)
    return users

In [22]:
def check_result(result_games: list, check_games: list):
    check_game_len = len(check_games)
    return (
        len(set(result_games[: len(check_games)]).intersection(check_games))
        / check_game_len
    )


In [23]:
def update_cbr_for_user(user, **current_parameters):
    p = ParametersManager(
        "cbr_for_user",
        current_parameters,
        {
            "min_player_count": 19,
            "cbr_for_game_result_count": 3,
            "max_game": 16,
        },
    )
    if p.is_diff_last_current():
        q = User.update({User.cbr_update_time:None}).where(User.cbr_update_time != None).execute()
    if days_delta(User.get_by_id(user.id).cbr_update_time) >= 1 or not user.cbr:
        # print(f'update cbr for "{user.username}"')
        update_cbr_for_game(min_player_count=p["min_player_count"])
        user.cbr = None
        last_played = np.quantile(
            [
                ug.last_played
                for ug in UserGame.select(UserGame.last_played).where(
                    (UserGame.user == user) & (UserGame.playtime > 0)
                )
            ],
            0.8,
        )
        played_games = (
            Game.select(Game.id, Game.cbr, UserGame.score)
            .join(UserGame)
            .where(
                (UserGame.user == user)
                & (UserGame.playtime > 0)
                & (UserGame.last_played < last_played)
            )
        )
        games_with_score = played_games.where((UserGame.score > 0) & (Game.cbr != None))
        # use normalized playtimes if not enough games with score
        if games_with_score.count() < 8:
            users_games_playtimes = get_normalized_playtimes(
                True,
                min_player_count=p["min_player_count"],
                zscore=False,
            )
            user_games_playtimes = (
                users_games_playtimes[str(user.id)]
                if str(user.id) in users_games_playtimes
                else []
            )
            result = []
            for game in played_games:
                if str(game.id) in user_games_playtimes.keys():
                    if game.cbr:
                        result.append(
                            {
                                "id": game.id,
                                "cbr": game.cbr,
                                "score": user_games_playtimes[str(game.id)],
                            }
                        )
                games_with_score = result
        # else use score
        else:
            games_with_score = games_with_score.dicts()
        games_with_score = sorted(
            games_with_score, key=lambda x: x["score"], reverse=True
        )[: p["max_game"]]
        # calc result
        result = []
        for game_a_dict in games_with_score:
            result.append(
                {
                    key: value * game_a_dict["score"]
                    for key, value in list(game_a_dict["cbr"].items())[
                        1 : p["cbr_for_game_result_count"] + 1
                    ]
                }
            )
        result = {
            str(game.id): value
            for game, value in get_readable_result_for_games(
                merge_dicts(result)
            ).items()
            if game not in played_games and game.rating >= 7
        }
        user.cbr = result
        user.cbr_update_time = dt.datetime.now()
        user.save()


def train_cbr_for_user(try_count=100):
    best_accuracy = 0
    best_min_player_count = 1
    best_cbr_for_game_result_count = 1
    max_game = 1
    file = Path("cbr.csv")
    if file.exists():
        df = pd.read_csv(file)
        data = df.values.tolist()
        data = sorted(data, key=lambda x: x[3], reverse=True)
        best = data[0]
        best_min_player_count = best[0]
        best_cbr_for_game_result_count = best[1]
        best_max_game = best[2]
        best_accuracy = best[3]
    else:
        data = []
    # print(data)

    def check(min_player_count, cbr_for_game_result_count, max_game):
        sub_data = [el[:3] for el in data]
        if [
            float(min_player_count),
            float(cbr_for_game_result_count),
            float(max_game),
        ] in sub_data:
            return True

    for i in range(try_count):
        min_player_count = random.randint(22, 23)
        cbr_for_game_result_count = random.randint(2, 3)
        max_game = random.randint(20, 32)
        if check(min_player_count, cbr_for_game_result_count, max_game):
            print("already exist")
            continue
        cbr_accuracy = []
        for user in get_users():
            played_user_games = UserGame.select(
                UserGame.game, UserGame.last_played
            ).where((UserGame.user == user) & (UserGame.playtime > 0))
            last_played = np.quantile([ug.last_played for ug in played_user_games], 0.8)
            input_user_games = played_user_games.where(
                UserGame.last_played < last_played
            )
            check_user_games = played_user_games.where(
                UserGame.last_played >= last_played
            )
            played_games = [ug.game for ug in input_user_games]
            check_games = [ug.game for ug in check_user_games]
            update_cbr_for_user(
                user,
                min_player_count=min_player_count,
                cbr_for_game_result_count=cbr_for_game_result_count,
                max_game=max_game,
            )
            cbr_result = get_readable_result_for_games(user.cbr, -1)
            cbr_accuracy.append(check_result(list(cbr_result.keys()), check_games))
        current_accuracy = np.mean(cbr_accuracy)
        data.append(
            [
                min_player_count,
                cbr_for_game_result_count,
                max_game,
                current_accuracy,
            ]
        )
        df = pd.DataFrame(data)
        df.to_csv(file, index=False)
        print(
            "current",
            current_accuracy,
            min_player_count,
            cbr_for_game_result_count,
            max_game,
        )
        if best_accuracy < current_accuracy:
            best_accuracy = current_accuracy
            best_min_player_count = min_player_count
            best_cbr_for_game_result_count = cbr_for_game_result_count
            best_max_game = max_game
            p = Parameters.get(name="cbr_for_user")
            p.best = {
                "min_player_count": best_min_player_count,
                "cbr_for_game_result_count": best_cbr_for_game_result_count,
                "max_game": best_max_game,
            }
            p.save()
        print(
            "best",
            best_accuracy,
            best_min_player_count,
            best_cbr_for_game_result_count,
            best_max_game,
        )


In [24]:
train_cbr_for_user(10)

already exist
update cbr for game
current 0.16743697478991598 22 3 21
best 0.1716386554621848 23.0 3.0 32.0
already exist
already exist
current 0.17457983193277313 22 2 30
best 0.17457983193277313 22 2 30
already exist
already exist
already exist
current 0.17163865546218487 22 3 24
best 0.17457983193277313 22 2 30
update cbr for game
current 0.1745798319327731 23 2 29
best 0.17457983193277313 22 2 30


In [25]:
# mbcf for user

def update_mbcf_for_user(user, **current_parameters):
    p = ParametersManager(
        "mbcf_for_user",
        current_parameters,
        {
            "sim_user_count": 10,
            "min_player_count": 10,
            "min_game_count": 10,
            "max_game": 10,
        },
    )
    if p.is_diff_last_current():
        q = User.update({User.mbcf_update_time:None}).where(User.mbcf_update_time != None).execute()
    if days_delta(User.get_by_id(user.id).mbcf_update_time) >= 1 or not user.mbcf:
        update_similar_users(min_game_count=p["min_game_count"], min_player_count=p["min_player_count"])
        if not user.similar_users:
            print("Fuck")
            return
        last_played = np.quantile(
            [
                ug.last_played
                for ug in UserGame.select(UserGame.last_played).where(
                    (UserGame.user == user) & (UserGame.playtime > 0)
                )
            ],
            0.8,
        )
        played_games = (
            Game.select(Game.id, Game.cbr, UserGame.score)
            .join(UserGame)
            .where(
                (UserGame.user == user)
                & (UserGame.playtime > 0)
                & (UserGame.last_played < last_played)
            )
        )
        normalized_playtimes = get_normalized_playtimes(
            True, min_player_count=p["min_player_count"], zscore=False
        )
        result = []
        similar_users = slice_dict(user.similar_users, 1, p["sim_user_count"] + 1)
        
        for user_id, coef in similar_users.items():
            if user_id in normalized_playtimes:
                result.append(
                    {
                        key: value * coef
                        for key, value in sorted(
                            list(normalized_playtimes[user_id].items()),
                            key=lambda x: x[1],
                            reverse=True,
                        )[: p["max_game"]]
                    }
                )
        user.mbcf = {
            str(game.id): value
            for game, value in get_readable_result_for_games(
                merge_dicts(result)
            ).items()
            if game not in played_games and game.rating >= 7
        }
        user.mbcf_update_time = dt.datetime.now()
        user.save()


def train_mbcf_for_user(try_count=100):
    best_accuracy = 0
    best_sim_user_count = 1
    best_min_player_count = 1
    best_min_game_count = 1
    best_max_game = 1
    file = Path("mbcf.csv")
    if file.exists():
        df = pd.read_csv(file)
        data = df.values.tolist()
        data = sorted(data, key=lambda x: x[4], reverse=True)
        best = data[0]
        best_sim_user_count = best[0]
        best_min_player_count = best[1]
        best_min_game_count = best[2]
        best_max_game = best[3]
        best_accuracy = best[4]
    else:
        data = []

    def check(sim_user_count, min_player_count, min_game_count, max_game):
        sub_data = [el[:4] for el in data]
        if [
            float(sim_user_count),
            float(min_player_count),
            float(min_game_count),
            float(max_game),
        ] in sub_data:
            # return False
            return True 

    for i in range(try_count):
        try:
            sim_user_count = random.randint(24-2, 39+2)
            min_player_count = random.randint(17-2, 21+2)
            min_game_count = random.randint(12-2, 26+2)
            max_game = random.randint(25-2, 36+2)
            # sim_user_count = 34
            # min_player_count = 16
            # min_game_count = 22
            # max_game = 31
            if check(sim_user_count, min_player_count, min_game_count, max_game):
                print("already exist")
                continue
            mbcf_accuracy = []
            for user in get_users():
                played_user_games = UserGame.select(
                    UserGame.game, UserGame.last_played
                ).where((UserGame.user == user) & (UserGame.playtime > 0))
                last_played = np.quantile(
                    [ug.last_played for ug in played_user_games], 0.8
                )
                input_user_games = played_user_games.where(
                    UserGame.last_played < last_played
                )
                check_user_games = played_user_games.where(
                    UserGame.last_played >= last_played
                )
                played_games = [ug.game for ug in input_user_games]
                check_games = [ug.game for ug in check_user_games]
                update_mbcf_for_user(
                    user,
                    sim_user_count=sim_user_count,
                    min_player_count=min_player_count,
                    min_game_count=min_game_count,
                    max_game=max_game,
                )
                mbcf_result = get_readable_result_for_games(user.mbcf, -1)
                mbcf_accuracy.append(
                    check_result(list(mbcf_result.keys()), check_games)
                )
            current_accuracy = np.mean(mbcf_accuracy)
            data.append(
                [
                    sim_user_count,
                    min_player_count,
                    min_game_count,
                    max_game,
                    current_accuracy,
                ]
            )
            df = pd.DataFrame(data)
            df.to_csv(file, index=False)
            print(
                "current",
                current_accuracy,
                sim_user_count,
                min_player_count,
                min_game_count,
                max_game,
            )
            if best_accuracy < current_accuracy:
                best_accuracy = current_accuracy
                best_sim_user_count = sim_user_count
                best_min_player_count = min_player_count
                best_min_game_count = min_game_count
                best_max_game = max_game
            p = Parameters.get(name="mbcf_for_user")
            p.best = {
                "sim_user_count": best_sim_user_count,
                "min_player_count": best_min_player_count,
                "min_game_count": best_min_game_count,
                "max_game": best_max_game,
            }
            p.save()
            print(
                "best",
                best_accuracy,
                best_sim_user_count,
                best_min_player_count,
                best_min_game_count,
                best_max_game,
            )
        except ValueError:
            print("Fuck")
            continue


In [26]:
train_mbcf_for_user(5)

update similar users
update normalized playtimes
current 0.2224628312863607 41 23 17 28
best 0.2224628312863607 41 23 17 28
update similar users
update normalized playtimes
current 0.22666451195862963 35 22 12 28
best 0.22666451195862963 35 22 12 28
update similar users
update normalized playtimes
current 0.19480715363068304 39 19 28 38
best 0.22666451195862963 35 22 12 28
update similar users
update normalized playtimes
current 0.1876642964878259 33 17 10 24
best 0.22666451195862963 35 22 12 28
update similar users
current 0.1876642964878259 30 17 13 32
best 0.22666451195862963 35 22 12 28


In [27]:
# mobcf for user
def update_mobcf_for_user(user, **current_parameters):
    p = ParametersManager(
        "mobcf_for_user",
        current_parameters,
        {"min_player_count": 20, "min_game_count": 10, "zscore": True},
    )
    if p.is_diff_last_current():
        q = User.update({User.mobcf_update_time:None}).where(User.mobcf_update_time != None).execute()
    if days_delta(User.get_by_id(user.id).mobcf_update_time) >= 1 or not user.mobcf:
        update_mobcf(
            min_player_count=p["min_player_count"],
            zscore=p["zscore"],
            min_game_count=p["min_game_count"],
        )
        with (user_data_dir / "userid2idx.json").open() as data_file:
            userid2idx = {int(k): v for k, v in json.load(data_file).items()}
        with (user_data_dir / "gameid2idx.json").open() as data_file:
            gameid2idx = {int(k): v for k, v in json.load(data_file).items()}
        num_users = len(userid2idx)
        num_items = len(gameid2idx)
        model = MF(num_users, num_items, emb_size=200).cuda()
        model.load_state_dict(torch.load(user_data_dir / "model.dat"))
        model.eval()
        if user.id in userid2idx:
            users = torch.LongTensor([userid2idx[user.id]]).cuda()
            games = list(gameid2idx.values())
            items = torch.LongTensor(games).cuda()
            result = model(users, items)
            idx2gameid = {value: key for key, value in gameid2idx.items()}
            result = {
                idx2gameid[game_idx.item()]: score.item()
                for score, game_idx in zip(result, items)
            }
            last_played = np.quantile(
                [
                    ug.last_played
                    for ug in UserGame.select(UserGame.last_played).where(
                        (UserGame.user == user) & (UserGame.playtime > 0)
                    )
                ],
                0.8,
            )
            played_games = [
                ug.game
                for ug in UserGame.select().where(
                    (UserGame.user == user)
                    & (UserGame.playtime > 0)
                    & (UserGame.last_played < last_played)
                )
            ]
            games = {
                Game.get_by_id(key): value
                for key, value in sorted(
                    result.items(), key=lambda item: item[1], reverse=True
                )
            }
            user.mobcf = {
                game.id: score
                for game, score in games.items()
                if game not in played_games
                and game.player_count > p["min_player_count"]
                and game.rating >= 7
            }
            user.mobcf_update_time = dt.datetime.now()
            user.save()


def train_mobcf_for_user(try_count=100):
    best_accuracy = -1
    best_min_player_count = 1
    best_min_game_count = 1
    best_zscore = True
    file = Path("mobcf.csv")
    if file.exists():
        df = pd.read_csv(file)
        data = df.values.tolist()
        data = sorted(data, key=lambda x: x[3], reverse=True)
        best = data[0]
        best_min_player_count = best[0]
        best_min_game_count = best[1]
        best_zscore = best[2]
        best_accuracy = best[3]
    else:
        data = []

    def check(min_player_count, min_game_count, zscore):
        sub_data = [el[:3] for el in data]
        if [min_player_count, min_game_count, zscore] in sub_data:
            return True

    for i in range(try_count):
        try:
            min_player_count = random.randint(10-2, 35)
            min_game_count = random.randint(5, 38)
            zscore = random.choice([True, False])
            if check(min_player_count, min_game_count, zscore):
                print("already exist")
                continue
            mobcf_accuracy = []
            for user in get_users():
                played_user_games = UserGame.select(
                    UserGame.game, UserGame.last_played
                ).where((UserGame.user == user) & (UserGame.playtime > 0))
                last_played = np.quantile(
                    [ug.last_played for ug in played_user_games], 0.8
                )
                input_user_games = played_user_games.where(
                    UserGame.last_played < last_played
                )
                check_user_games = played_user_games.where(
                    UserGame.last_played >= last_played
                )
                played_games = [ug.game for ug in input_user_games]
                check_games = [ug.game for ug in check_user_games]
                update_mobcf_for_user(
                    user,
                    min_player_count=min_player_count,
                    min_game_count=min_game_count,
                    zscore=zscore
                )
                mobcf_result = get_readable_result_for_games(user.mobcf, -1)
                mobcf_accuracy.append(
                    check_result(list(mobcf_result.keys()), check_games)
                )
            current_accuracy = np.mean(mobcf_accuracy)
            data.append(
                [min_player_count, min_game_count, zscore, current_accuracy]
            )
            df = pd.DataFrame(data)
            df.to_csv(file, index=False)
            print(
                "current",
                current_accuracy,
                min_player_count,
                min_game_count,
                zscore,
            )
            if best_accuracy < current_accuracy:
                best_accuracy = current_accuracy
                best_min_player_count = min_player_count
                best_min_game_count = min_game_count
                best_zscore = zscore
            p = Parameters.get(name="mobcf_for_user")
            p.best = {
                "min_player_count": best_min_player_count,
                "min_game_count": best_min_game_count,
                "zscore": best_zscore,
            }
            p.save()
            print(
                "best",
                best_accuracy,
                best_min_player_count,
                best_min_game_count,
                best_zscore,
            )
        except ValueError:
            continue


In [32]:
train_mobcf_for_user(20)

update mobcf
update normalized playtimes
train loss 0.000 valid loss 6.264
train loss 0.000 valid loss 0.019
train loss 0.000 valid loss 0.015
train loss 0.000 valid loss 0.015
current 0.044392372333548806 24 16 False
best 0.0685574229691876 20 19 False
update mobcf
update normalized playtimes
train loss 0.000 valid loss 7.485
train loss 0.000 valid loss 0.992
train loss 0.000 valid loss 1.030
train loss 0.000 valid loss 1.042
current 0.13276233570351217 22 14 True
best 0.13276233570351217 22 14 True
update mobcf
update normalized playtimes
train loss 0.000 valid loss 4.127
train loss 0.000 valid loss 0.013
train loss 0.000 valid loss 0.013
train loss 0.000 valid loss 0.013
current 0.0899859943977591 29 7 False
best 0.13276233570351217 22 14 True
update mobcf
update normalized playtimes
train loss 0.000 valid loss 3.596
train loss 0.000 valid loss 1.118
train loss 0.000 valid loss 1.142
train loss 0.000 valid loss 1.150
current 0.13723874165050634 8 26 True
best 0.13723874165050634 8 2

In [29]:
# hrs for game

def update_hr_for_user(user, **current_parameters):
    p = ParametersManager(
        "hr_for_user",
        current_parameters,
        {"cbr_coef": 0.5, "mbcf_coef": 0.5, "mobcf_coef": 0.5},
    )
    if p.is_diff_last_current():
        q = User.update({User.hr_update_time:None}).where(User.hr_update_time != None).execute()
    if days_delta(User.get_by_id(user.id).hr_update_time) >= 1 or not user.hr:
        update_cbr_for_user(user)
        update_mbcf_for_user(user)
        update_mobcf_for_user(user)
        user.hr = merge_dicts(
            [
                normalize_dict(user.cbr, p["cbr_coef"]) if user.cbr else [],
                normalize_dict(user.mbcf, p["mbcf_coef"]) if user.mbcf else [],
                normalize_dict(user.mobcf, p["mobcf_coef"]) if user.mobcf else [],
            ]
        )
        user.hr_update_time = dt.datetime.now()
        user.save()

def train_hrs_for_user(try_count = 100):
    best_accuracy = -1
    best_cbr_coef = 0.01
    best_mbcf_coef = 0.01
    best_mobcf_coef = 0.01
    file = Path("hrs.csv")
    if file.exists():
        df = pd.read_csv(file)
        data = df.values.tolist()
    else:
        data = []
    def check(cbr_coef, mbcf_coef, mobcf_coef):
        sub_data = [el[:3] for el in data]
        if [cbr_coef, mbcf_coef, mobcf_coef] in sub_data:
            return True
    for i in range(try_count):
        try:
            cbr_coef= random.randint(1, 100) / 100.0
            # cbr_coef= 0.88
            mbcf_coef = random.randint(1, 100) / 100.0
            # mbcf_coef = 0.73
            mobcf_coef = random.randint(1, 100) / 100.0
            # mobcf_coef = 0.03
            if check(cbr_coef, mbcf_coef, mobcf_coef):
                print("already exist")
                continue
            hrs_accuracy = []
            for user in get_users():
                played_user_games = UserGame.select(
                    UserGame.game, UserGame.last_played
                ).where((UserGame.user == user) & (UserGame.playtime > 0))
                last_played = np.quantile(
                    [ug.last_played for ug in played_user_games], 0.8
                )
                input_user_games = played_user_games.where(
                    UserGame.last_played < last_played
                )
                check_user_games = played_user_games.where(
                    UserGame.last_played >= last_played
                )
                played_games = [ug.game for ug in input_user_games]
                check_games = [ug.game for ug in check_user_games]
                update_hr_for_user(
                    user, cbr_coef=cbr_coef, mbcf_coef=mbcf_coef, mobcf_coef =mobcf_coef
                )
                hrs_result = get_readable_result_for_games(user.hr, -1)
                hrs_accuracy.append(check_result(list(hrs_result.keys()), check_games))
            current_accuracy = np.mean(hrs_accuracy)
            data.append([cbr_coef, mbcf_coef, mobcf_coef, current_accuracy])
            df = pd.DataFrame(data)
            df.to_csv(f20ers.get(name="hr_for_user")
            p.best = {
                "cbr_coef": best_cbr_coef,
                "mbcf_coef": best_mbcf_coef,
                "mobcf_coef": best_mobcf_coef,
            }
            p.save()
            if best_accuracy < current_accuracy:
                best_accuracy = current_accuracy
                best_cbr_coef = cbr_coef
                best_mbcf_coef = mbcf_coef
                best_mobcf_coef = mobcf_coef
            print("best", best_accuracy, best_cbr_coef, best_mbcf_coef, best_mobcf_coef)
        except ValueError:
            continue

In [33]:
train_hrs_for_user(6)

update mobcf
update normalized playtimes
train loss 0.000 valid loss 3.793
train loss 0.000 valid loss 1.080
train loss 0.000 valid loss 1.107
train loss 0.000 valid loss 1.115
current 0.25082956259426853 0.74 0.73 0.91
best 0.25082956259426853 0.74 0.73 0.91
current 0.17659448394742513 0.22 0.83 0.5
best 0.25082956259426853 0.74 0.73 0.91
current 0.19781835811247575 0.98 0.5 0.51
best 0.25082956259426853 0.74 0.73 0.91
current 0.175759534583064 0.3 0.15 0.4
best 0.25082956259426853 0.74 0.73 0.91
current 0.19361667744020686 0.4 0.76 0.82
best 0.25082956259426853 0.74 0.73 0.91
current 0.21531997414350354 0.01 0.71 0.04
best 0.25082956259426853 0.74 0.73 0.91
current 0.13703404438698555 0.63 0.04 0.6
best 0.25082956259426853 0.74 0.73 0.91
current 0.16560547295841413 0.44 0.85 0.52
best 0.25082956259426853 0.74 0.73 0.91
current 0.19361667744020686 0.66 0.94 0.57
best 0.25082956259426853 0.74 0.73 0.91
current 0.2216278819219996 0.86 0.51 0.46
best 0.25082956259426853 0.74 0.73 0.91
cu

KeyboardInterrupt: 

In [31]:
# benchmark
cbr_accuracy = []
mbcf_accuracy = []
mobcf_accuracy = []
hrs_accuracy = []

for user in get_users():
    played_user_games = UserGame.select(UserGame.game, UserGame.last_played).where((UserGame.user == user) & (UserGame.playtime > 0))
    last_played = np.quantile([ug.last_played for ug in played_user_games], 0.8)
    input_user_games = played_user_games.where(UserGame.last_played < last_played)
    check_user_games = played_user_games.where(UserGame.last_played >= last_played)
    played_games = [ug.game for ug in input_user_games]
    check_games = [ug.game for ug in check_user_games]
    update_cbr_for_user(user)
    update_mbcf_for_user(user)
    update_mobcf_for_user(user)
    update_hr_for_user(user)
    cbr_result = get_readable_result_for_games(user.cbr)
    mbcf_result = get_readable_result_for_games(user.mbcf)
    mobcf_result = get_readable_result_for_games(user.mobcf)
    hrs_result = get_readable_result_for_games(user.hr)
    cbr_accuracy.append(check_result(list(cbr_result.keys()), check_games))
    mbcf_accuracy.append(check_result(list(mbcf_result.keys()), check_games))
    mobcf_accuracy.append(check_result(list(mobcf_result.keys()), check_games))
    hrs_accuracy.append(check_result(list(hrs_result.keys()), check_games))
print("cbr accuracy:", np.mean(cbr_accuracy))
print("mbcf accuracy:", np.mean(mbcf_accuracy))
print("mobcf accuracy:", np.mean(mobcf_accuracy))
print("hrs accuracy:", np.mean(hrs_accuracy))

cbr accuracy: 0.1745798319327731
mbcf accuracy: 0.2114738202973497
mobcf accuracy: 0.06855742296918767
hrs accuracy: 0.2508295625942685
